In [ ]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

HOME: /content


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
%%capture
!pip install open_clip_torch

In [ ]:
%cd {HOME}
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO
!git checkout -q 57535c5a79791cb76e36fdb64975271354f10251
!pip install -q -e .

/content
fatal: destination path 'GroundingDINO' already exists and is not an empty directory.
/content/GroundingDINO
  Preparing metadata (setup.py) ... done


In [ ]:
%cd {HOME}

import sys
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

/content
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-yund96p7
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-yund96p7
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done


In [ ]:
!pip uninstall -y supervision
!pip install -q supervision==0.6.0

import supervision as sv
print(sv.__version__)

Found existing installation: supervision 0.4.0
Uninstalling supervision-0.4.0:
  Successfully uninstalled supervision-0.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
groundingdino 0.1.0 requires supervision==0.4.0, but you have supervision 0.6.0 which is incompatible.
0.6.0


In [ ]:
import os

GROUNDING_DINO_CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(GROUNDING_DINO_CONFIG_PATH, "; exist:", os.path.isfile(GROUNDING_DINO_CONFIG_PATH))

/content/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py ; exist: True


In [ ]:
%cd {HOME}
!mkdir -p {HOME}/weights
%cd {HOME}/weights

!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

/content
/content/weights


In [ ]:
GROUNDING_DINO_CHECKPOINT_PATH = os.path.join(HOME, "weights", "groundingdino_swint_ogc.pth")
print(GROUNDING_DINO_CHECKPOINT_PATH, "; exist:", os.path.isfile(GROUNDING_DINO_CHECKPOINT_PATH))

/content/weights/groundingdino_swint_ogc.pth ; exist: True


In [ ]:
%cd {HOME}
!mkdir -p {HOME}/weights
%cd {HOME}/weights

!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

/content
/content/weights


In [ ]:
SAM_CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(SAM_CHECKPOINT_PATH, "; exist:", os.path.isfile(SAM_CHECKPOINT_PATH))

/content/weights/sam_vit_h_4b8939.pth ; exist: True


In [ ]:
%cd {HOME}/GroundingDINO

from groundingdino.util.inference import Model

grounding_dino_model = Model(model_config_path=GROUNDING_DINO_CONFIG_PATH, model_checkpoint_path=GROUNDING_DINO_CHECKPOINT_PATH)

/content/GroundingDINO


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


In [ ]:
from segment_anything import sam_model_registry, SamPredictor

SAM_ENCODER_VERSION = "vit_h"

sam = sam_model_registry[SAM_ENCODER_VERSION](checkpoint=SAM_CHECKPOINT_PATH).to('cuda')
sam_predictor = SamPredictor(sam)

In [ ]:
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
import torchvision.transforms as T
from torchvision.io import read_image
from torch.utils.data import Dataset
import pandas as pd
import torchvision.transforms as T
import torchvision.transforms.functional as F
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import random
import open_clip
from typing import List
import math
import cv2
import copy
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%%capture
!unzip /content/gdrive/MyDrive/test-archive.zip -d /content/testing-dataset

In [ ]:
!mv "/content/testing-dataset/development_test_data/gallery" "/content/"
!mv "/content/testing-dataset/development_test_data/queries" "/content/"

mv: cannot move '/content/testing-dataset/development_test_data/gallery' to '/content/gallery': Directory not empty
mv: cannot move '/content/testing-dataset/development_test_data/queries' to '/content/queries': Directory not empty


In [ ]:
!mkdir "/content/original_datasets"
!mv "/content/testing-dataset/development_test_data/gallery.csv" "/content/original_datasets/"
!mv "/content/testing-dataset/development_test_data/queries.csv" "/content/original_datasets/"

mkdir: cannot create directory ‘/content/original_datasets’: File exists


In [ ]:
!rm -rf "/content/testing-dataset"

In [ ]:
!mkdir '/content/image_to_text_results/'
!cp '/content/gdrive/MyDrive/prompt_What-is-the-product-category_all_version.csv' '/content/image_to_text_results/'
!cp '/content/gdrive/MyDrive/prompt_What-is-the-product-category_gallery_version.csv' '/content/image_to_text_results/'
!cp '/content/gdrive/MyDrive/prompt_What-is-the-product-category.csv' '/content/image_to_text_results/'

mkdir: cannot create directory ‘/content/image_to_text_results/’: File exists


In [ ]:
%%capture
!unzip '/content/gdrive/MyDrive/gallery_tensors_blip_detection_bbox_crop_bt_0.4_tt_0.25.zip' -d '/'
!unzip '/content/gdrive/MyDrive/query_tensors_blip_detection_bbox_crop_bt_0.4_tt_0.25.zip' -d '/'

In [ ]:
!mkdir '/content/pretrained'
!cp '/content/gdrive/MyDrive/visual-product-recognition-pre-trained/model1.pt' '/content/pretrained/'
!cp '/content/gdrive/MyDrive/visual-product-recognition-pre-trained/model2.pt' '/content/pretrained/'
!cp '/content/gdrive/MyDrive/visual-product-recognition-pre-trained/model3.pt' '/content/pretrained/'
!cp '/content/gdrive/MyDrive/visual-product-recognition-pre-trained/model4.pt' '/content/pretrained/'

mkdir: cannot create directory ‘/content/pretrained’: File exists


In [ ]:
masked_similarity = pd.read_csv('/content/gallery_tensors_blip_detection_bbox_crop_bt_0.4_tt_0.25/gallery_tensors_blip_detection_bbox_crop_bt_0.4_tt_0.25.csv', low_memory=False)
masked_similarity.head()

,index,image_tensor_path,masked_image_tensor_path,inverse_masked_image_tensor_path,label,image_path,masked_similarity,inverse_masked_similarity
0,0,gallery_tensors_blip_detection_bbox_crop_bt_0....,gallery_tensors_blip_detection_bbox_crop_bt_0....,gallery_tensors_blip_detection_bbox_crop_bt_0....,5019,gallery/ambitious-tough-teal-from-asgard.jpg,12.038039,18.042643
1,1,gallery_tensors_blip_detection_bbox_crop_bt_0....,gallery_tensors_blip_detection_bbox_crop_bt_0....,gallery_tensors_blip_detection_bbox_crop_bt_0....,5008,gallery/fine-shrewd-oarfish-of-genius.jpg,5.544218,18.558239
2,2,gallery_tensors_blip_detection_bbox_crop_bt_0....,gallery_tensors_blip_detection_bbox_crop_bt_0....,gallery_tensors_blip_detection_bbox_crop_bt_0....,9,gallery/bold-nickel-gecko-of-reputation.jpg,1.164761,14.268379
3,3,gallery_tensors_blip_detection_bbox_crop_bt_0....,gallery_tensors_blip_detection_bbox_crop_bt_0....,gallery_tensors_blip_detection_bbox_crop_bt_0....,3589,gallery/marvellous-uber-boobook-of-lightning.jpg,4.094361,12.754694
4,4,gallery_tensors_blip_detection_bbox_crop_bt_0....,gallery_tensors_blip_detection_bbox_crop_bt_0....,gallery_tensors_blip_detection_bbox_crop_bt_0....,3656,gallery/bouncy-economic-agama-of-honeydew.jpg,2.484441,15.336771


In [ ]:
detection_category = pd.read_csv('/content/image_to_text_results/prompt_What-is-the-product-category_all_version.csv', low_memory=False)
detection_category.head()

,index,label,category,count
0,0,5019,shoes,7
1,1,5008,shoes,8
2,2,9,basketball,5
3,3,3589,sunglasses,3
4,4,3656,sunglasses,4


In [ ]:
gallery = pd.read_csv("/content/original_datasets/gallery.csv", low_memory=False)
gallery.head()

,seller_img_id,img_path,product_id
0,0,gallery/ambitious-tough-teal-from-asgard.jpg,5019
1,1,gallery/fine-shrewd-oarfish-of-genius.jpg,5008
2,2,gallery/bold-nickel-gecko-of-reputation.jpg,9
3,3,gallery/marvellous-uber-boobook-of-lightning.jpg,3589
4,4,gallery/bouncy-economic-agama-of-honeydew.jpg,3656


In [ ]:
def get_mask_or_inverse_mask_to_use(img_path):
  ms = (masked_similarity.loc[masked_similarity['image_path'] == img_path, 'masked_similarity'].values)[0]
  ims = (masked_similarity.loc[masked_similarity['image_path'] == img_path, 'inverse_masked_similarity'].values)[0]
  if ms <= ims:
    return 1
  else:
    return 0

img_df = gallery
df = pd.DataFrame(columns=['img1', 'img2', 'label', 'img1_masked', 'img2_masked', 'img1_category', 'img2_category'])
product_ids = gallery['product_id'].unique()
row_count = 0
id_count = {1:0, 2:0, 3:0, 4:0}
for product_id in product_ids:
  df1 = img_df.loc[img_df['product_id'] == product_id]
  img_category = (detection_category.loc[detection_category['label'] == product_id, 'category'].values)[0]
  img_path_index = df1.columns.get_loc('img_path')
  if len(df1) == 2:
    row_count += 1
    id_count[2] += 1
    img1 = df1.iloc[0, img_path_index]
    img2 = df1.iloc[1, img_path_index]
    df.loc[len(df)] = [img1, img2, 1, get_mask_or_inverse_mask_to_use(img1), get_mask_or_inverse_mask_to_use(img2), img_category, img_category]
  elif len(df1) == 3:
    img1 = df1.iloc[0, img_path_index]
    img2 = df1.iloc[1, img_path_index]
    img3 = df1.iloc[2, img_path_index]
    row_count += 2
    id_count[3] += 1
    df.loc[len(df)] = [img1, img2, 1, get_mask_or_inverse_mask_to_use(img1), get_mask_or_inverse_mask_to_use(img2), img_category, img_category]
    df.loc[len(df)] = [img2, img3, 1, get_mask_or_inverse_mask_to_use(img2), get_mask_or_inverse_mask_to_use(img3), img_category, img_category]
  elif len(df1) > 3:
    id_count[4] += 1
    img1 = df1.iloc[0, img_path_index]
    img2 = df1.iloc[1, img_path_index]
    img3 = df1.iloc[2, img_path_index]
    img4 = df1.iloc[3, img_path_index]
    row_count += 2
    df.loc[len(df)] = [img1, img2, 1, get_mask_or_inverse_mask_to_use(img1), get_mask_or_inverse_mask_to_use(img2), img_category, img_category]
    df.loc[len(df)] = [img3, img4, 1, get_mask_or_inverse_mask_to_use(img3), get_mask_or_inverse_mask_to_use(img4), img_category, img_category]
  else:
    id_count[1] += 1
  df2 = img_df.loc[img_df['product_id'] != product_id]
  if len(df1) < 4:
    img1 = df1.iloc[0, img_path_index]
    count = 0
    for index, row in df2.iterrows():
      if count >= 4:
        break
      if ((df['img1'] == img1) & (df['img2'] == row['img_path'])).any() or ((df['img1'] == row['img_path']) & (df['img2'] == img1)).any():
        continue
      else:
        img2 = row['img_path']
        row_count += 1
        img2_category = (detection_category.loc[detection_category['label'] == row['product_id'], 'category'].values)[0]
        df.loc[len(df)] = [img1, img2, 0, get_mask_or_inverse_mask_to_use(img1), get_mask_or_inverse_mask_to_use(img2), img_category, img2_category]
        count += 1
  else:
    img1 = df1.iloc[0, img_path_index]
    img2 = df1.iloc[1, img_path_index]
    count = 0
    for index, row in df2.iterrows():
      if count >= 4:
        break
      if count >= 2:
        img1 = img2
      if ((df['img1'] == img1) & (df['img2'] == row['img_path'])).any() or ((df['img1'] == row['img_path']) & (df['img2'] == img1)).any():
        continue
      else:
        img3 = row['img_path']
        row_count += 1
        img2_category = (detection_category.loc[detection_category['label'] == row['product_id'], 'category'].values)[0]
        df.loc[len(df)] = [img1, img3, 0, get_mask_or_inverse_mask_to_use(img1), get_mask_or_inverse_mask_to_use(img3), img_category, img2_category]
        count += 1

df.to_csv('/content/train_validation_dataset.csv')
print('row_count: ', row_count)
print(len(df.loc[df['label'] == 1]))
print(id_count)

row_count:  2871
435
{1: 284, 2: 215, 3: 90, 4: 20}


In [ ]:
print('shape: ', df.shape)
display(df.head())

shape:  (2871, 7)


,img1,img2,label,img1_masked,img2_masked,img1_category,img2_category
0,gallery/ambitious-tough-teal-from-asgard.jpg,gallery/busy-aspiring-oryx-of-peace.jpg,1,1,1,shoes,shoes
1,gallery/busy-aspiring-oryx-of-peace.jpg,gallery/vigorous-upbeat-barracuda-of-strength.jpg,1,1,1,shoes,shoes
2,gallery/ambitious-tough-teal-from-asgard.jpg,gallery/fine-shrewd-oarfish-of-genius.jpg,0,1,1,shoes,shoes
3,gallery/ambitious-tough-teal-from-asgard.jpg,gallery/bold-nickel-gecko-of-reputation.jpg,0,1,1,shoes,basketball
4,gallery/ambitious-tough-teal-from-asgard.jpg,gallery/marvellous-uber-boobook-of-lightning.jpg,0,1,1,shoes,sunglasses


In [ ]:
df1 = df.loc[df['label'] == 1]
df2 = df.loc[df['label'] == 0]

msk1 = np.random.rand(len(df1)) < 0.8
train1 = df1[msk1]
validation1 = df1[~msk1]

msk2 = np.random.rand(len(df2)) < 0.8
train2 = df2[msk2]
validation2 = df2[~msk2]

train = pd.concat([train1, train2], axis=0)
validation = pd.concat([validation1, validation2], axis=0)

train.reset_index(inplace=True)
train.drop(['index'], inplace=True, axis=1)
train.to_csv('/content/train_dataset_for_segmentation_model.csv')

validation.reset_index(inplace=True)
validation.drop(['index'], inplace=True, axis=1)
validation.to_csv('/content/validation_dataset_for_segmentation_model.csv')

print('Train\n====================================')
print('shape: ', train.shape)
display(train.head())

print('Validation\n====================================')
print('shape: ', validation.shape)
display(validation.head())

Train
shape:  (2302, 7)


,img1,img2,label,img1_masked,img2_masked,img1_category,img2_category
0,gallery/ambitious-tough-teal-from-asgard.jpg,gallery/busy-aspiring-oryx-of-peace.jpg,1,1,1,shoes,shoes
1,gallery/busy-aspiring-oryx-of-peace.jpg,gallery/vigorous-upbeat-barracuda-of-strength.jpg,1,1,1,shoes,shoes
2,gallery/fine-shrewd-oarfish-of-genius.jpg,gallery/alluring-quiet-peacock-of-renovation.jpg,1,1,1,shoes,shoes
3,gallery/dainty-stork-of-hypothetical-health.jpg,gallery/kind-opalescent-lion-of-adventure.jpg,1,1,1,shoes,shoes
4,gallery/bold-nickel-gecko-of-reputation.jpg,gallery/colorful-mutant-boobook-of-popularity.jpg,1,1,1,basketball,basketball


Validation
shape:  (569, 7)


,img1,img2,label,img1_masked,img2_masked,img1_category,img2_category
0,gallery/abiding-debonair-viper-of-downpour.jpg,gallery/modest-proud-ant-of-triumph.jpg,1,1,1,shoes,shoes
1,gallery/first-clay-mule-of-aurora.jpg,gallery/judicious-evasive-deer-of-purring.jpg,1,0,1,shoes,shoes
2,gallery/burrowing-carmine-shrew-of-growth.jpg,gallery/tacky-berserk-earwig-of-luxury.jpg,1,1,1,shoes,shoes
3,gallery/fearless-cherry-sawfly-of-sunshine.jpg,gallery/benign-ludicrous-aardwark-of-karma.jpg,1,1,1,sunglasses,sunglasses
4,gallery/perky-mutant-dachshund-of-witchcraft.jpg,gallery/vegan-sparkling-dragonfly-of-performan...,1,0,1,shoes,shoes


In [ ]:
!cp '/content/train_dataset_for_segmentation_model.csv' '/content/gdrive/MyDrive/'
!cp '/content/validation_dataset_for_segmentation_model.csv' '/content/gdrive/MyDrive/'

In [ ]:
def enhance_class_name(class_names: List[str]) -> List[str]:
    return [
        f"all {class_name}s"
        for class_name
        in class_names
    ]

def segment(sam_predictor: SamPredictor, image: np.ndarray, xyxy: np.ndarray) -> np.ndarray:
    sam_predictor.set_image(image)
    result_masks = []
    for box in xyxy:
        masks, scores, logits = sam_predictor.predict(
            box=box,
            multimask_output=True
        )
        index = np.argmax(scores)
        result_masks.append(masks[index])
    return np.array(result_masks)

def detect_masks(img_path, category):
    if category is not None:
      CLASSES = [category]
    else:
      CLASSES = ['products']
    BOX_TRESHOLD = 0.4
    TEXT_TRESHOLD = 0.25
    image = cv2.imread(img_path)
    detections = grounding_dino_model.predict_with_classes(
        image=image,
        classes=enhance_class_name(class_names=CLASSES),
        box_threshold=BOX_TRESHOLD,
        text_threshold=TEXT_TRESHOLD
    )
    detections = detections[detections.class_id != None]
    detections.mask = segment(
        sam_predictor=sam_predictor,
        image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB),
        xyxy=detections.xyxy
    )
    return detections.mask, detections.xyxy

def get_mask_and_inverse_mask(img_path, category):
  masks, bboxes = detect_masks(img_path, category)
  image = read_image(img_path)

  if len(masks) >= 1:
    masks_np_arr = {}

    for i in range(len(masks)):
      mask = masks[i:i+1, :]
      mask = mask.astype(int)
      mask = np.repeat(mask, 3, axis = 0)
      # mask = np.where(mask==0, 0, 1)
      # mask = mask.astype('uint8')
      masks_np_arr[i] = mask

    final_mask = None
    if len(masks_np_arr) > 1:
      for key, mask in masks_np_arr.items():
        if final_mask is None:
          final_mask = mask
        else:
          final_mask = np.add(final_mask, mask)
      final_mask = np.where(final_mask > 0, 1, 0)
      final_inverse_mask = np.where(final_mask==0, 1, 0)
      # final_mask = final_mask.astype('uint8')
      # final_inverse_mask = final_inverse_mask.astype('uint8')
    else:
      final_mask = masks_np_arr[0]
      final_inverse_mask = np.where(final_mask==0, 1, 0)
      # final_inverse_mask = final_inverse_mask.astype('uint8')

    image_ = image.numpy()

    # final_mask = final_mask.astype('int64')
    final_mask = np.where(final_mask == 0, -1, 1)
    masked = np.multiply(image_, final_mask)
    masked = np.where(masked < 0, 240, masked)
    masked = masked.astype('uint8')

    # final_inverse_mask = final_inverse_mask.astype('int64')
    final_inverse_mask = np.where(final_inverse_mask == 0, -1, 1)
    inverse_masked = np.multiply(image_, final_inverse_mask)
    inverse_masked = np.where(inverse_masked < 0, 240, inverse_masked)
    inverse_masked = inverse_masked.astype('uint8')

    masked = torch.tensor(masked)
    inverse_masked = torch.tensor(inverse_masked)

    # calculate bounding box parameters for cropping
    x = []
    y = []
    for bbox in bboxes:
      x.append(bbox[0])
      x.append(bbox[2])
      y.append(bbox[1])
      y.append(bbox[3])
    min_x = math.floor(min(x))
    min_y = math.floor(min(y))
    max_x = math.ceil(max(x))
    max_y = math.ceil(max(y))

    return image, masked, inverse_masked, (min_x, min_y, max_x - min_x, max_y - min_y)
  else:
    return image, None, None, None

In [ ]:
def get_img_color_convert(img_path):
  """
  This function returns an image in the format RGB when the path
  to the image is given

  Parameters
  ----------
  img_path: string
      The path to the image

  Returns
  -------
  RGB image of the image in the specified path
  """
  return cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

def get_img(img_path):
  """
  This function returns an image in the format RGB when the path
  to the image is given

  Parameters
  ----------
  img_path: string
      The path to the image

  Returns
  -------
  RGB image of the image in the specified path
  """
  return cv2.imread(img_path)

def draw_bounding_box_and_save(img_path, x, y, w, h):
  """
  This function draw a bounding box specified by x, y, w, h parameters
  and save the image in the '/content/temp-imgs/' folder

  Parameters
  ----------
  img_path: string
      The path to original image which the bounding box should be drawn
  x: integer
      x-coordinate of top-left corner of bounding box
  y: integer
      y-coordinate of top-left corner of bounding box
  w: integer
      width of bounding box in pixels
  h: integer
      height of bounding box in pixels
  """
  image = get_img(img_path)
  image = image[y:y+h, x:x+w]
  # cv2.rectangle(image, (x, y), (x + w, y + h), (0,255,255), 4)

  Path("/content/temp-imgs").mkdir(parents=True, exist_ok=True)

  filepath = '/content/temp-imgs/' + img_path.split('/')[-1]
  cv2.imwrite(filepath, image)
  return filepath

In [ ]:
img_transform = T.Compose([T.ToPILImage(),
                           T.Resize(size=(224, 224),
                                    interpolation=T.InterpolationMode.BICUBIC,
                                    antialias=True),
                           T.ToTensor(),
                           T.Normalize(mean=(0.48145466, 0.4578275, 0.40821073),
                                       std=(0.26862954, 0.26130258, 0.27577711))])

In [ ]:
class TrainValidationDataset(Dataset):
    def __init__(self, annotations_file, img_dir, indices, save_folder,
                 transform=None,
                 crop_transform=None,
                 target_transform=None,
                 crop_to_bbox=False,
                 crop_to_bbox_detections=False,
                 mode='save'):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.indices = indices
        self.transform = transform
        self.target_transform = target_transform
        self.crop_to_bbox = crop_to_bbox
        self.crop_to_bbox_detections = crop_to_bbox_detections
        self.crop_transform = crop_transform
        self.mode = mode
        self.save_folder = save_folder

        folder_name1 = f'train/gallery'
        if not os.path.exists('/content/' + folder_name1):
          os.makedirs('/content/' + folder_name1)

        folder_name2 = f'validate/gallery'
        if not os.path.exists('/content/' + folder_name2):
          os.makedirs('/content/' + folder_name2)

    def __len__(self):
        return len(self.img_labels)
        # return 2

    def __getitem__(self, idx):
        img1_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, self.indices[0]])
        img2_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, self.indices[1]])

        if self.mode == 'save':
          label = self.img_labels.iloc[idx, self.indices[2]]
          img1_mask = self.img_labels.iloc[idx, self.indices[3]]
          img2_mask = self.img_labels.iloc[idx, self.indices[4]]
          img1_cat = self.img_labels.iloc[idx, self.indices[5]]
          img2_cat = self.img_labels.iloc[idx, self.indices[6]]

          if pd.isna(img1_cat):
            img1_cat = None
          if pd.isna(img2_cat):
            img2_cat = None

          masked1_file = Path(f'/content/{self.save_folder}/'+ self.img_labels.iloc[idx, self.indices[0]] + '.pt')
          masked2_file = Path(f'/content/{self.save_folder}/'+ self.img_labels.iloc[idx, self.indices[1]] + '.pt')

          if masked1_file.exists() and masked2_file.exists():
            return 0
          else:
            if not masked1_file.exists():
              image1, masked1, inverse_masked1, bbox1 = get_mask_and_inverse_mask('/content/' + img1_path, img1_cat)
            if not masked2_file.exists():
              image2, masked2, inverse_masked2, bbox2 = get_mask_and_inverse_mask('/content/' + img2_path, img2_cat)

            if not masked1_file.exists() and img1_mask == 0:
              masked1 = inverse_masked1

            if not masked2_file.exists() and img2_mask == 0:
              masked2 = inverse_masked2

            if self.transform and not self.crop_to_bbox_detections:
                image1 = self.transform(image1)
                image2 = self.transform(image2)
                if not masked1_file.exists() and masked1 is not None:
                  masked1 = self.transform(masked1)
                if not masked2_file.exists() and masked2 is not None:
                  masked2 = self.transform(masked2)
            elif self.crop_transform and self.crop_to_bbox_detections:
                if not masked1_file.exists() and masked1 is not None and bbox1 is not None:
                  x = bbox1[0]
                  y = bbox1[1]
                  w = bbox1[2]
                  h = bbox1[3]
                  image1 = self.crop_transform(image1, x, y, w, h)
                  masked1 = self.crop_transform(masked1, x, y, w, h)
                elif not masked1_file.exists():
                  if self.transform:
                    image1 = self.transform(image1)

                if not masked2_file.exists() and masked2 is not None and bbox2 is not None:
                  x = bbox2[0]
                  y = bbox2[1]
                  w = bbox2[2]
                  h = bbox2[3]
                  image2 = self.crop_transform(image2, x, y, w, h)
                  masked2 = self.crop_transform(masked2, x, y, w, h)
                elif not masked2_file.exists():
                  if self.transform:
                    image2 = self.transform(image2)

            if self.target_transform:
                label = self.target_transform(label)

            if not masked1_file.exists() and masked1 is None:
              masked1 = image1

            if not masked2_file.exists() and masked2 is None:
              masked2 = image2

            num_saved = 0
            masked1_file = Path(f'/content/{self.save_folder}/'+ self.img_labels.iloc[idx, self.indices[0]] + '.pt')
            masked2_file = Path(f'/content/{self.save_folder}/'+ self.img_labels.iloc[idx, self.indices[1]] + '.pt')
            if not masked1_file.exists():
              num_saved += 1
              torch.save(masked1, f'/content/{self.save_folder}/'+ self.img_labels.iloc[idx, self.indices[0]] + '.pt')
            if not masked2_file.exists():
              num_saved += 1
              torch.save(masked2, f'/content/{self.save_folder}/'+ self.img_labels.iloc[idx, self.indices[1]] + '.pt')

            return num_saved
        else:
          masked1 = torch.load(img1_path + '.pt')
          masked2 = torch.load(img2_path + '.pt')
          label = self.img_labels.iloc[idx, self.indices[2]]

          return masked1, masked2, label

def image_transformation(image):
    images = img_transform(image)
    #images = img_transform(F.to_pil_image(image))
    return images

def image_transform_with_bbox_crop(image, x, y, w, h):
    images = F.crop(image, y, x, h, w)
    images = img_transform(images)
    return images

In [ ]:
!rm -rf '/content/train'
!rm -rf '/content/validate'

In [ ]:
train_dataset = TrainValidationDataset("/content/train_dataset_for_segmentation_model.csv", "", [1, 2, 3, 4, 5, 6, 7], 'train', transform=image_transformation, crop_transform=image_transform_with_bbox_crop, crop_to_bbox_detections=True)
validation_dataset = TrainValidationDataset("/content/validation_dataset_for_segmentation_model.csv", "", [1, 2, 3, 4, 5, 6, 7], 'validate', transform=image_transformation, crop_transform=image_transform_with_bbox_crop, crop_to_bbox_detections=True)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=32, shuffle=True)

count = 0
for i, num in enumerate(tqdm(train_dataloader)):
  count += np.sum(num.numpy())
print('number of unique saved images: ', count)

count = 0
for i, num in enumerate(tqdm(validation_dataloader)):
  count += np.sum(num.numpy())
print('number of unique saved images: ', count)

100%|██████████| 72/72 [14:15<00:00, 11.89s/it]


number of unique saved images:  994


100%|██████████| 18/18 [06:45<00:00, 22.53s/it]

number of unique saved images:  477


In [ ]:
%%capture
!zip -r '/content/train_segment.zip' '/content/train'

In [ ]:
%%capture
!zip -r '/content/validate_segment.zip' '/content/validate'

In [ ]:
!cp '/content/train_segment.zip' '/content/gdrive/MyDrive/train_segment.zip'
!cp '/content/validate_segment.zip' '/content/gdrive/MyDrive/validate_segment.zip'

In [ ]:
%%capture
!unzip '/content/gdrive/MyDrive/train_segment.zip' -d '/'

In [ ]:
%%capture
!unzip '/content/gdrive/MyDrive/validate_segment.zip' -d '/'

In [ ]:
!cp '/content/gdrive/MyDrive/train_dataset_for_segmentation_model.csv' '/content/'
!cp '/content/gdrive/MyDrive/validation_dataset_for_segmentation_model.csv' '/content/'

In [ ]:
del grounding_dino_model
del sam_predictor
del sam

torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

Wed Nov  1 16:22:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_model = None
        self.best_loss = None
        self.counter = 0
        self.status = ""

    def __call__(self, model, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
        elif self.best_loss - val_loss >= self.min_delta:
            self.best_model = copy.deepcopy(model.state_dict())
            self.best_loss = val_loss
            self.counter = 0
            self.status = f"Improvement found, counter reset to {self.counter}"
        else:
            self.counter += 1
            self.status = f"No improvement in the last {self.counter} epochs"
            if self.counter >= self.patience:
                self.status = f"Early stopping triggered after {self.counter} epochs."
                if self.restore_best_weights:
                    model.load_state_dict(self.best_model)
                return True
        return False

    def save_the_best_model(self, path):
      torch.save(self.best_model, path)
      return self.best_loss

In [ ]:
vit_model = open_clip.create_model_and_transforms('ViT-H-14', None)[0].visual
vit_model.load_state_dict(torch.load('/content/pretrained/model3.pt'))
vit_model

VisionTransformer(
  (conv1): Conv2d(3, 1280, kernel_size=(14, 14), stride=(14, 14), bias=False)
  (patch_dropout): Identity()
  (ln_pre): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
  (transformer): Transformer(
    (resblocks): ModuleList(
      (0-31): 32 x ResidualAttentionBlock(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=1280, out_features=1280, bias=True)
        )
        (ls_1): Identity()
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (c_fc): Linear(in_features=1280, out_features=5120, bias=True)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=5120, out_features=1280, bias=True)
        )
        (ls_2): Identity()
      )
    )
  )
  (ln_post): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
)

In [ ]:
for name, para in vit_model.named_parameters():
  print("-"*20)
  print(f"name: {name}")

--------------------
name: class_embedding
--------------------
name: positional_embedding
--------------------
name: proj
--------------------
name: conv1.weight
--------------------
name: ln_pre.weight
--------------------
name: ln_pre.bias
--------------------
name: transformer.resblocks.0.ln_1.weight
--------------------
name: transformer.resblocks.0.ln_1.bias
--------------------
name: transformer.resblocks.0.attn.in_proj_weight
--------------------
name: transformer.resblocks.0.attn.in_proj_bias
--------------------
name: transformer.resblocks.0.attn.out_proj.weight
--------------------
name: transformer.resblocks.0.attn.out_proj.bias
--------------------
name: transformer.resblocks.0.ln_2.weight
--------------------
name: transformer.resblocks.0.ln_2.bias
--------------------
name: transformer.resblocks.0.mlp.c_fc.weight
--------------------
name: transformer.resblocks.0.mlp.c_fc.bias
--------------------
name: transformer.resblocks.0.mlp.c_proj.weight
--------------------
name:

In [ ]:
# del vit_model
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

Wed Nov  1 16:23:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Fully connected neural network with one hidden layer
class SegmentationModel(nn.Module):
    def __init__(self):
        super(SegmentationModel, self).__init__()

        vit_model = open_clip.create_model_and_transforms('ViT-H-14', None)[0].visual
        vit_model.load_state_dict(torch.load('/content/pretrained/model3.pt'))
        self.vit_model = vit_model

    def forward(self, x1, x2):
      emb1 = self.vit_model(x1)
      emb2 = self.vit_model(x2)
      return emb1, emb2

In [ ]:
def criterion(x1, x2, label, margin: float = 35.0):
    """
    Computes Contrastive Loss
    """

    dist = torch.nn.functional.pairwise_distance(x1, x2, p=2)

    loss = (label) * torch.pow(dist, 2) + (1 - label) * torch.pow(torch.clamp(margin - dist, min=0.0), 2)
    loss = torch.mean(loss)

    return loss

In [ ]:
import os

# Make sure reproducability
np.random.seed(0)
torch.manual_seed(0)

# hyperparameters
num_epochs = 10
learning_rate = 0.01

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SegmentationModel().to(device)
for name, para in model.named_parameters():
  if 'mlp.c_proj' in name:
    para.requires_grad = True
  else:
    para.requires_grad = False

# Loss and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Early stopper initialization
es = EarlyStopping(patience=5)

# LR scheduler initialization
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)

# Initialize datasets and dataloaders
train_dataset = TrainValidationDataset("/content/train_dataset_for_segmentation_model.csv", "/content/train/", [1, 2, 3, 4, 5, 6, 7], '', transform=image_transformation, crop_transform=image_transform_with_bbox_crop, crop_to_bbox_detections=True, mode='load')
validation_dataset = TrainValidationDataset("/content/validation_dataset_for_segmentation_model.csv", "/content/validate/", [1, 2, 3, 4, 5, 6, 7], '', transform=image_transformation, crop_transform=image_transform_with_bbox_crop, crop_to_bbox_detections=True, mode='load')
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=2, shuffle=True)

# Gather data for graphing
train_loss_arr = []
test_loss_arr = []

# Train the model
n_total_steps_train = len(train_dataloader)
n_total_steps_validation = len(validation_dataloader)
# Data structure to hold the best values
best_model_on_loss_value = {'train_loss':1000000, 'test_loss':1000000, 'epoch':0}
best_model_on_accuracy_value = {'train_loss':1000000, 'test_loss':1000000, 'epoch':0}
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}')
    avg_train_loss = 0
    for i, (imgs1, imgs2, labels) in enumerate(tqdm(train_dataloader)):
        imgs1, imgs2, labels = imgs1.to('cuda'), imgs2.to('cuda'), labels.to('cuda')

        # Forward pass
        embs1, embs2 = model(imgs1, imgs2)
        loss = criterion(embs1, embs2, labels)

        # print(f'Epoch {epoch+1} Batch {i + 1}, Training Loss: {loss:.6f}')

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_train_loss += loss.item()
    print('\n============================================================================================')
    print(f'Epoch {epoch+1}, Training Loss: {(avg_train_loss / n_total_steps_train):.6f}')
    print('============================================================================================\n')

    # Validate the model
    # In test phase, we don't need to compute gradients (for memory efficiency)
    with torch.no_grad():
        avg_val_loss = 0
        n_correct = 0
        n_samples = 0
        for imgs1, imgs2, labels in tqdm(validation_dataloader):
            imgs1, imgs2, labels = imgs1.to('cuda'), imgs2.to('cuda'), labels.to('cuda')

            # Forward pass
            embs1, embs2 = model(imgs1, imgs2)
            loss = criterion(embs1, embs2, labels)

            avg_val_loss += loss

        scheduler.step(avg_val_loss / n_total_steps_validation)

        print('\n============================================================================================')
        print(f'Epoch {epoch+1}, Validation Loss: {(avg_val_loss / n_total_steps_validation):.6f}')
        print('============================================================================================\n')

        train_loss_arr.append(avg_train_loss / n_total_steps_train)
        test_loss_arr.append(avg_val_loss / n_total_steps_validation)

        if es(model, (avg_val_loss / n_total_steps_validation)):
            print('Early stopping criteria reached. Training stopped.')
            break

Epoch 1


100%|██████████| 1151/1151 [06:52<00:00,  2.79it/s]



Epoch 1, Training Loss: 867.207379



100%|██████████| 285/285 [00:38<00:00,  7.33it/s]



Epoch 1, Validation Loss: 343.763153

Epoch 2


...


Epoch 9, Validation Loss: 230.947723

Epoch 10


100%|██████████| 1151/1151 [07:03<00:00,  2.72it/s]



Epoch 10, Training Loss: 179.690862



100%|██████████| 285/285 [00:39<00:00,  7.25it/s]


Epoch 10, Validation Loss: 189.027847

Early stopping criteria reached. Training stopped.


In [ ]:
# es.save_the_best_model('/content/pretrained/model4.pt')

In [ ]:
torch.save(es.best_model, '/content/pretrained/model4.pt')

In [ ]:
!cp '/content/pretrained/model4.pt' '/content/gdrive/MyDrive/visual-product-recognition-pre-trained/'

In [ ]:
print(es.best_loss)

tensor(182.8780, device='cuda:0')


In [ ]:
del model
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

Wed Nov  1 18:07:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    39W / 300W |   9076MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Test the model

In [ ]:
!mkdir '/content/image_to_text_results/'
!cp '/content/gdrive/MyDrive/prompt_What-is-the-product-category_all_version.csv' '/content/image_to_text_results/'
!cp '/content/gdrive/MyDrive/prompt_What-is-the-product-category_gallery_version.csv' '/content/image_to_text_results/'
!cp '/content/gdrive/MyDrive/prompt_What-is-the-product-category.csv' '/content/image_to_text_results/'

mkdir: cannot create directory ‘/content/image_to_text_results/’: File exists


In [ ]:
CLASSES = ['products']
BOX_TRESHOLD = 0.4
TEXT_TRESHOLD = 0.25

def enhance_class_name(class_names: List[str]) -> List[str]:
    return [
        f"all {class_name}s"
        for class_name
        in class_names
    ]

def segment(sam_predictor: SamPredictor, image: np.ndarray, xyxy: np.ndarray) -> np.ndarray:
    sam_predictor.set_image(image)
    result_masks = []
    for box in xyxy:
        masks, scores, logits = sam_predictor.predict(
            box=box,
            multimask_output=True
        )
        index = np.argmax(scores)
        result_masks.append(masks[index])
    return np.array(result_masks)

def detect_masks(img_path, category):
    if category is not None:
      CLASSES = [category]
    else:
      CLASSES = ['products']
    image = cv2.imread(img_path)
    detections = grounding_dino_model.predict_with_classes(
        image=image,
        classes=enhance_class_name(class_names=CLASSES),
        box_threshold=BOX_TRESHOLD,
        text_threshold=TEXT_TRESHOLD
    )
    detections = detections[detections.class_id != None]
    detections.mask = segment(
        sam_predictor=sam_predictor,
        image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB),
        xyxy=detections.xyxy
    )
    return detections.mask, detections.xyxy

def get_mask_and_inverse_mask(img_path, category):
  masks, bboxes = detect_masks(img_path, category)
  image = read_image(img_path)

  if len(masks) >= 1:
    masks_np_arr = {}

    for i in range(len(masks)):
      mask = masks[i:i+1, :]
      mask = mask.astype(int)
      mask = np.repeat(mask, 3, axis = 0)
      # mask = np.where(mask==0, 0, 1)
      # mask = mask.astype('uint8')
      masks_np_arr[i] = mask

    final_mask = None
    if len(masks_np_arr) > 1:
      for key, mask in masks_np_arr.items():
        if final_mask is None:
          final_mask = mask
        else:
          final_mask = np.add(final_mask, mask)
      final_mask = np.where(final_mask > 0, 1, 0)
      final_inverse_mask = np.where(final_mask==0, 1, 0)
      # final_mask = final_mask.astype('uint8')
      # final_inverse_mask = final_inverse_mask.astype('uint8')
    else:
      final_mask = masks_np_arr[0]
      final_inverse_mask = np.where(final_mask==0, 1, 0)
      # final_inverse_mask = final_inverse_mask.astype('uint8')

    image_ = image.numpy()

    # final_mask = final_mask.astype('int64')
    final_mask = np.where(final_mask == 0, -1, 1)
    masked = np.multiply(image_, final_mask)
    masked = np.where(masked < 0, 240, masked)
    masked = masked.astype('uint8')

    # final_inverse_mask = final_inverse_mask.astype('int64')
    final_inverse_mask = np.where(final_inverse_mask == 0, -1, 1)
    inverse_masked = np.multiply(image_, final_inverse_mask)
    inverse_masked = np.where(inverse_masked < 0, 240, inverse_masked)
    inverse_masked = inverse_masked.astype('uint8')

    masked = torch.tensor(masked)
    inverse_masked = torch.tensor(inverse_masked)

    # calculate bounding box parameters for cropping
    x = []
    y = []
    for bbox in bboxes:
      x.append(bbox[0])
      x.append(bbox[2])
      y.append(bbox[1])
      y.append(bbox[3])
    min_x = math.floor(min(x))
    min_y = math.floor(min(y))
    max_x = math.ceil(max(x))
    max_y = math.ceil(max(y))

    return image, masked, inverse_masked, (min_x, min_y, max_x - min_x, max_y - min_y)
  else:
    return image, None, None, None

In [ ]:
img_transform = T.Compose([T.ToPILImage(),
                           T.Resize(size=(224, 224),
                                    interpolation=T.InterpolationMode.BICUBIC,
                                    antialias=True),
                           T.ToTensor(),
                           T.Normalize(mean=(0.48145466, 0.4578275, 0.40821073),
                                       std=(0.26862954, 0.26130258, 0.27577711))])

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, annotations_file, category_annotations_file, img_dir, indices,
                 transform=None,
                 crop_transform=None,
                 target_transform=None,
                 crop_to_bbox=False,
                 crop_to_bbox_detections=False):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_categories = pd.read_csv(category_annotations_file)
        self.img_dir = img_dir
        self.indices = indices
        self.transform = transform
        self.target_transform = target_transform
        self.crop_to_bbox = crop_to_bbox
        self.crop_to_bbox_detections = crop_to_bbox_detections
        self.crop_transform = crop_transform

    def __len__(self):
        return len(self.img_labels)
        # return 2

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, self.indices[0]])
        label = self.img_labels.iloc[idx, self.indices[1]]
        category = self.img_categories.loc[self.img_categories.label == 49, 'category'].values[0]
        if pd.isna(category):
          category = None
        if self.crop_to_bbox:
          x = self.img_labels.iloc[idx, self.indices[2]]
          y = self.img_labels.iloc[idx, self.indices[3]]
          w = self.img_labels.iloc[idx, self.indices[4]]
          h = self.img_labels.iloc[idx, self.indices[5]]
          cropped_img_path = draw_bounding_box_and_save('/content/' + img_path, x, y, w, h)
          image, masked, inverse_masked, bbox = get_mask_and_inverse_mask(cropped_img_path, category)
        else:
          image, masked, inverse_masked, bbox = get_mask_and_inverse_mask('/content/' + img_path, category)
        if self.transform and not self.crop_to_bbox_detections:
            image = self.transform(image)
            if masked is not None and inverse_masked is not None:
              masked = self.transform(masked)
              inverse_masked = self.transform(inverse_masked)
        elif self.crop_transform and self.crop_to_bbox_detections:
            if masked is not None and inverse_masked is not None and bbox is not None:
              x = bbox[0]
              y = bbox[1]
              w = bbox[2]
              h = bbox[3]
              image = self.crop_transform(image, x, y, w, h)
              masked = self.crop_transform(masked, x, y, w, h)
              inverse_masked = self.crop_transform(inverse_masked, x, y, w, h)
            else:
              if self.transform:
                image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        if masked is None and inverse_masked is None:
          masked = torch.empty(0)
          inverse_masked = torch.empty(0)
        return image, masked, inverse_masked, label, img_path

def image_transformation(image):
    images = img_transform(image)
    #images = img_transform(F.to_pil_image(image))
    return images

def image_transform_with_bbox_crop(image, x, y, w, h):
    images = F.crop(image, y, x, h, w)
    images = img_transform(images)
    return images

In [ ]:
def get_img_color_convert(img_path):
  """
  This function returns an image in the format RGB when the path
  to the image is given

  Parameters
  ----------
  img_path: string
      The path to the image

  Returns
  -------
  RGB image of the image in the specified path
  """
  return cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

def get_img(img_path):
  """
  This function returns an image in the format RGB when the path
  to the image is given

  Parameters
  ----------
  img_path: string
      The path to the image

  Returns
  -------
  RGB image of the image in the specified path
  """
  return cv2.imread(img_path)

def draw_bounding_box_and_save(img_path, x, y, w, h):
  """
  This function draw a bounding box specified by x, y, w, h parameters
  and save the image in the '/content/temp-imgs/' folder

  Parameters
  ----------
  img_path: string
      The path to original image which the bounding box should be drawn
  x: integer
      x-coordinate of top-left corner of bounding box
  y: integer
      y-coordinate of top-left corner of bounding box
  w: integer
      width of bounding box in pixels
  h: integer
      height of bounding box in pixels
  """
  image = get_img(img_path)
  image = image[y:y+h, x:x+w]
  # cv2.rectangle(image, (x, y), (x + w, y + h), (0,255,255), 4)

  Path("/content/temp-imgs").mkdir(parents=True, exist_ok=True)

  filepath = '/content/temp-imgs/' + img_path.split('/')[-1]
  cv2.imwrite(filepath, image)
  return filepath

def get_query_and_gallery_images_for_a_product_id(product_id):
  """
  This function returns a list of gallery image paths and a list of query
  image paths correcponding to a specific product_id in testing dataset

  Parameters
  ----------
  product_id: integer

  Returns
  -------
  Two list of file paths of gallery and query images corresponding to product_id
  """
  query_images_for_product_id = query.loc[query['product_id'] == product_id]
  query_image_list = []
  for index, row in query_images_for_product_id.iterrows():
    img_path = row['img_path']
    x = row['bbox_x']
    y = row['bbox_y']
    w = row['bbox_w']
    h = row['bbox_h']
    query_image_list.append(draw_bounding_box_and_save('/content/' + img_path, x, y, w, h))

  gallery_images_for_product_id = gallery.loc[gallery['product_id'] == product_id]
  gallery_image_list = gallery_images_for_product_id['img_path'].tolist()

  return query_image_list, gallery_image_list

In [ ]:
vit_model = open_clip.create_model_and_transforms('ViT-H-14', None)[0].visual
vit_model.load_state_dict(torch.load('/content/pretrained/model3.pt'))
vit_model.load_state_dict(torch.load('/content/pretrained/model4.pt'), strict=False)
vit_model.eval()

VisionTransformer(
  (conv1): Conv2d(3, 1280, kernel_size=(14, 14), stride=(14, 14), bias=False)
  (patch_dropout): Identity()
  (ln_pre): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
  (transformer): Transformer(
    (resblocks): ModuleList(
      (0-31): 32 x ResidualAttentionBlock(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=1280, out_features=1280, bias=True)
        )
        (ls_1): Identity()
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (c_fc): Linear(in_features=1280, out_features=5120, bias=True)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=5120, out_features=1280, bias=True)
        )
        (ls_2): Identity()
      )
    )
  )
  (ln_post): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
)

In [ ]:
vit_model = vit_model.to('cuda')

In [ ]:
gallery_dataset = CustomDataset("/content/original_datasets/gallery.csv", "/content/image_to_text_results/prompt_What-is-the-product-category_all_version.csv", "", [1, 2], transform=image_transformation, crop_transform=image_transform_with_bbox_crop, crop_to_bbox_detections=True)
gallery_dataloader = DataLoader(gallery_dataset, batch_size=1, shuffle=False)

df = pd.DataFrame(columns=['image_tensor_path', 'masked_image_tensor_path', 'inverse_masked_image_tensor_path', 'label', 'image_path', 'masked_similarity', 'inverse_masked_similarity'])

folder_name = f'gallery_tensors_blip_detection_bbox_crop_trained_bt_{BOX_TRESHOLD}_tt_{TEXT_TRESHOLD}'
if not os.path.exists('/content/' + folder_name):
    os.makedirs('/content/' + folder_name)

# graph_index = 1
i = 0
for idx, data in enumerate(tqdm(gallery_dataloader)):
    image, masked, inverse_masked, label, img_path = data

    # fig = plt.figure(figsize=(8, 8))

    # proc_img = F.to_pil_image(masked[0])
    # plt.subplot(2, 2, graph_index), plt.imshow(proc_img)

    # proc_img = F.to_pil_image(inverse_masked[0])
    # plt.subplot(2, 2, graph_index + 1), plt.imshow(proc_img)

    # graph_index += 2

    image = image.to('cuda')

    if masked.nelement() != 0 and masked.nelement() != 0:
      masked = masked.to('cuda')
      inverse_masked = inverse_masked.to('cuda')

    with torch.no_grad():
        output_image = vit_model(image)
        embeddings_image = output_image.cpu()
        if masked.nelement() != 0 and masked.nelement() != 0:
          output_masked_image = vit_model(masked)
          output_inverse_masked_image = vit_model(inverse_masked)
          embeddings_masked_image = output_masked_image.cpu()
          embeddings_inverse_masked_image = output_inverse_masked_image.cpu()

    file_path_image = f'{folder_name}/image_{idx}_{i}.pt'
    torch.save(embeddings_image[0], '/content/' + file_path_image)
    if masked.nelement() != 0 and masked.nelement() != 0:
      file_path_masked_image = f'{folder_name}/masked_image_{idx}_{i}.pt'
      file_path_inverse_masked_image = f'{folder_name}/inverse_masked_image_{idx}_{i}.pt'
      torch.save(embeddings_masked_image[0], '/content/' + file_path_masked_image)
      torch.save(embeddings_inverse_masked_image[0], '/content/' + file_path_inverse_masked_image)

      euclidean_masked = torch.nn.functional.pairwise_distance(embeddings_image, embeddings_masked_image, p=2)
      euclidean_inverse_masked = torch.nn.functional.pairwise_distance(embeddings_image, embeddings_inverse_masked_image, p=2)

      df.loc[len(df)] = [file_path_image, file_path_masked_image, file_path_inverse_masked_image, label.item(), img_path[0], euclidean_masked.item(), euclidean_inverse_masked.item()]
    else:
      df.loc[len(df)] = [file_path_image, 'None', 'None', label.item(), img_path[0], -1, -1]

df.reset_index(inplace = True)
df.to_csv(f'/content/{folder_name}/gallery_tensors_blip_detection_bbox_crop_trained_bt_{BOX_TRESHOLD}_tt_{TEXT_TRESHOLD}.csv', index=False)

100%|██████████| 1067/1067 [17:54<00:00,  1.01s/it]


In [ ]:
query_dataset = CustomDataset("/content/original_datasets/queries.csv", "/content/image_to_text_results/prompt_What-is-the-product-category_all_version.csv", "", [1, 6, 2, 3, 4, 5], transform=image_transformation, crop_transform=image_transform_with_bbox_crop, crop_to_bbox_detections=True, crop_to_bbox=True)
query_dataloader = DataLoader(query_dataset, batch_size=1, shuffle=False)

df = pd.DataFrame(columns=['image_tensor_path', 'masked_image_tensor_path', 'inverse_masked_image_tensor_path', 'label', 'image_path', 'masked_similarity', 'inverse_masked_similarity'])

folder_name = f'query_tensors_blip_detection_bbox_crop_trained_bt_{BOX_TRESHOLD}_tt_{TEXT_TRESHOLD}'
if not os.path.exists('/content/' + folder_name):
    os.makedirs('/content/' + folder_name)

# graph_index = 1
for idx, data in enumerate(tqdm(query_dataloader)):
    image, masked, inverse_masked, label, img_path = data

    # fig = plt.figure(figsize=(8, 8))

    # proc_img = F.to_pil_image(masked[0])
    # plt.subplot(2, 2, graph_index), plt.imshow(proc_img)

    # proc_img = F.to_pil_image(inverse_masked[0])
    # plt.subplot(2, 2, graph_index + 1), plt.imshow(proc_img)

    # graph_index += 2

    image = image.to('cuda')
    if masked.nelement() != 0 and masked.nelement() != 0:
      masked = masked.to('cuda')
      inverse_masked = inverse_masked.to('cuda')

    with torch.no_grad():
        output_image = vit_model(image)
        embeddings_image = output_image.cpu()
        if masked.nelement() != 0 and masked.nelement() != 0:
          output_masked_image = vit_model(masked)
          output_inverse_masked_image = vit_model(inverse_masked)
          embeddings_masked_image = output_masked_image.cpu()
          embeddings_inverse_masked_image = output_inverse_masked_image.cpu()

    file_path_image = f'{folder_name}/image_{idx}_{i}.pt'
    torch.save(embeddings_image[0], '/content/' + file_path_image)
    if masked.nelement() != 0 and masked.nelement() != 0:
      file_path_masked_image = f'{folder_name}/masked_image_{idx}_{i}.pt'
      file_path_inverse_masked_image = f'{folder_name}/inverse_masked_image_{idx}_{i}.pt'
      torch.save(embeddings_masked_image[0], '/content/' + file_path_masked_image)
      torch.save(embeddings_inverse_masked_image[0], '/content/' + file_path_inverse_masked_image)

      euclidean_masked = torch.nn.functional.pairwise_distance(embeddings_image, embeddings_masked_image, p=2)
      euclidean_inverse_masked = torch.nn.functional.pairwise_distance(embeddings_image, embeddings_inverse_masked_image, p=2)

      df.loc[len(df)] = [file_path_image, file_path_masked_image, file_path_inverse_masked_image, label.item(), img_path[0], euclidean_masked.item(), euclidean_inverse_masked.item()]
    else:
      df.loc[len(df)] = [file_path_image, 'None', 'None', label.item(), img_path[0], -1, -1]

df.reset_index(inplace = True)
df.to_csv(f'/content/{folder_name}/query_tensors_blip_detection_bbox_crop_trained_bt_{BOX_TRESHOLD}_tt_{TEXT_TRESHOLD}.csv', index=False)

100%|██████████| 1935/1935 [31:21<00:00,  1.03it/s]


In [ ]:
%%capture
!zip -r '/content/gallery_tensors_blip_detection_bbox_crop_trained_bt_0.4_tt_0.25.zip' '/content/gallery_tensors_blip_detection_bbox_crop_bt_0.4_tt_0.25/'

In [ ]:
%%capture
!zip -r '/content/query_tensors_blip_detection_bbox_crop_trained_bt_0.4_tt_0.25.zip' '/content/query_tensors_blip_detection_bbox_crop_bt_0.4_tt_0.25/'

In [ ]:
!cp '/content/gallery_tensors_blip_detection_bbox_crop_trained_bt_0.4_tt_0.25.zip' '/content/gdrive/MyDrive/gallery_tensors_blip_detection_bbox_crop_trained_bt_0.4_tt_0.25.zip'
!cp '/content/query_tensors_blip_detection_bbox_crop_trained_bt_0.4_tt_0.25.zip' '/content/gdrive/MyDrive/query_tensors_blip_detection_bbox_crop_trained_bt_0.4_tt_0.25.zip'

In [ ]:
!unzip '/content/gdrive/MyDrive/gallery_tensors_blip_detection_bbox_crop_trained_bt_0.4_tt_0.25.zip' -d '/'
!unzip '/content/gdrive/MyDrive/query_tensors_blip_detection_bbox_crop_trained_bt_0.4_tt_0.25.zip' -d '/'

In [ ]:
gallery_tensor_dataset_path = '/content/gallery_tensors_blip_detection_bbox_crop_trained_bt_0.4_tt_0.25/gallery_tensors_blip_detection_bbox_crop_trained_bt_0.4_tt_0.25.csv'
query_tensor_dataset_path = '/content/query_tensors_blip_detection_bbox_crop_trained_bt_0.4_tt_0.25/query_tensors_blip_detection_bbox_crop_trained_bt_0.4_tt_0.25.csv'

In [ ]:
class TensorDataset(Dataset):
    def __init__(self, annotations_file, tensor_dir, transform=None, target_transform=None):
        self.tensor_labels = pd.read_csv(annotations_file)
        self.tensor_dir = tensor_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.tensor_labels)

    def __getitem__(self, idx):
        image_tensor_path = os.path.join(self.tensor_dir, self.tensor_labels.iloc[idx, 1])
        masked_tensor_path = os.path.join(self.tensor_dir, self.tensor_labels.iloc[idx, 2])
        inverse_masked_tensor_path = os.path.join(self.tensor_dir, self.tensor_labels.iloc[idx, 3])

        image = torch.load(image_tensor_path)
        if self.tensor_labels.iloc[idx, 2] != 'None' and self.tensor_labels.iloc[idx, 3] != 'None':
          masked = torch.load(masked_tensor_path)
          inverse_masked = torch.load(inverse_masked_tensor_path)
        else:
          masked = torch.empty(0)
          inverse_masked = torch.empty(0)

        label = self.tensor_labels.iloc[idx, 4]
        image_path = self.tensor_labels.iloc[idx, 5]

        euclidian_masked = self.tensor_labels.iloc[idx, 6]
        euclidian_inverse_masked = self.tensor_labels.iloc[idx, 7]

        if self.transform:
            tensor = self.transform(tensor)
        if self.target_transform:
            label = self.target_transform(label)
        return image, masked, inverse_masked, label, image_path, euclidian_masked, euclidian_inverse_masked

In [ ]:
tensor_dataset = TensorDataset(gallery_tensor_dataset_path, "/content/")
tensor_dataloader = DataLoader(tensor_dataset, batch_size=1, shuffle=False)

image_tensor_dict = {}
masked_tensor_dict = {}
label_dict = {}
image_path_dict = {}

for idx, data in enumerate(tqdm(tensor_dataloader)):
    image, masked, inverse_masked, label, image_path, euclidian_masked, euclidian_inverse_masked = data

    image_tensor_dict[idx] = image

    if masked.nelement() != 0 and inverse_masked.nelement() != 0:
      if euclidian_masked <= euclidian_inverse_masked:
        masked_tensor_dict[idx] = masked
      else:
        masked_tensor_dict[idx] = inverse_masked
    else:
      masked_tensor_dict[idx] = torch.empty(0)

    label_dict[idx] = label
    image_path_dict[idx] = image_path[0]

100%|██████████| 1067/1067 [00:01<00:00, 575.66it/s]


In [ ]:
query_tensor_dataset = TensorDataset(query_tensor_dataset_path, "/content/")
query_tensor_dataloader = DataLoader(query_tensor_dataset, batch_size=1, shuffle=False)

query_image_tensor_dict = {}
query_masked_tensor_dict = {}
query_label_dict = {}
query_image_path_dict = {}

for idx, data in enumerate(tqdm(query_tensor_dataloader)):
    image, masked, inverse_masked, label, image_path, euclidian_masked, euclidian_inverse_masked = data

    query_image_tensor_dict[idx] = image

    if masked.nelement() != 0 and inverse_masked.nelement() != 0:
      if euclidian_masked <= euclidian_inverse_masked:
        query_masked_tensor_dict[idx] = masked
      else:
        query_masked_tensor_dict[idx] = inverse_masked
    else:
      query_masked_tensor_dict[idx] = torch.empty(0)

    query_label_dict[idx] = label
    query_image_path_dict[idx] = image_path[0]

100%|██████████| 1935/1935 [00:02<00:00, 783.16it/s]


In [ ]:
MAX_VAL = 1000000

def AP(actual_label, predicted_labels):
    GPTs = predicted_labels.count(actual_label)
    if GPTs < 1:
        return 1
    AP = 0
    GPTs_found = 0
    for i in range(len(predicted_labels)):
        if predicted_labels[i] == actual_label:
            GPTs_found += 1
            AP += GPTs_found / (i + 1)
    return AP / GPTs

def mAP(APs):
    return np.average(APs)

def get_similarity_score(q, qm, g, gm):
    if gm.nelement() != 0 and qm.nelement() != 0:
      distance = torch.nn.functional.pairwise_distance(qm, gm, p=2)
    else:
      distance = torch.nn.functional.pairwise_distance(q, g, p=2)

    # distance = torch.nn.functional.pairwise_distance(q, g, p=2)

    return torch.tensor(distance.item())


def get_mAP_of_model(query_tensor_dictionary, query_masked_tensor_dictionary, query_label_dictionary):
    APs_euclidean = []
    APs_manhattan = []
    APs_cosine = []
    for i in tqdm(range(len(query_tensor_dictionary))):
        query_tensor = query_tensor_dictionary[i].to('cuda')
        query_masked_tensor = query_masked_tensor_dictionary[i].to('cuda')

        euclidean = None
        pred_labels = None
        act_label = query_label_dictionary[i]

        for k in range(len(image_tensor_dict)):
          gallery_tensor = image_tensor_dict[k].to('cuda')
          gallery_masked_tensor = masked_tensor_dict[k].to('cuda')
          gallery_label = label_dict[k]

          if euclidean == None:
            euclidean = torch.unsqueeze(get_similarity_score(query_tensor, query_masked_tensor, gallery_tensor, gallery_masked_tensor), 0)
            pred_labels = gallery_label
          else:
            distance = torch.unsqueeze(get_similarity_score(query_tensor, query_masked_tensor, gallery_tensor, gallery_masked_tensor), 0)
            euclidean = torch.cat((euclidean, distance), dim=0)
            pred_labels = torch.cat((pred_labels, gallery_label), dim=0)

        euclidean_, euclidean_indices = torch.sort(euclidean)

        pred_labels_euclidean = pred_labels[euclidean_indices]

        APs_euclidean.append(AP(act_label.item(), pred_labels_euclidean.cpu().numpy().astype(int).tolist()))

    print('\nEuclidean distance mAP:', mAP(APs_euclidean))

In [1]:
get_mAP_of_model(query_image_tensor_dict, query_masked_tensor_dict, query_label_dict)

100%|██████████| 1935/1935 [07:42<00:00,  4.18it/s]
Euclidean distance mAP: 0.6252792589434875


In [ ]:
del vit_model
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

Wed Nov  1 20:16:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    39W / 300W |   6238MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------